In [1]:
import pyspark
from operator import add
from pyspark import SparkConf
from pyspark.ml.feature import NGram
from pyspark.sql.functions import col,udf
from pyspark.sql import SQLContext
from operator import add
import numpy as np
from pyspark.mllib.linalg.distributed import CoordinateMatrix, MatrixEntry
import string
from pyspark.mllib.util import MLUtils
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.ml.feature import StringIndexer, VectorIndexer
sc = pyspark.SparkContext('local[*]',appName="DocClassification")
sqlc = SQLContext(sc)



In [6]:
def swap(x):
    return (x[1],x[0])

strs = sc.parallelize(["This is something I really want to test.",
                       "To test something is something I really want to do.",
                       "I need to do something that I want to do which is to test something."])
strs = strs.zipWithIndex()\
            .map(lambda x: swap(x))\
            .flatMapValues(lambda x: x.split())\
            .mapValues(lambda x: x.lower().strip(string.punctuation))\
            .map(lambda x: ((x[0],x[1]),1))

labs = sc.parallelize([(1,1),(2,0),(3,1)]).toDF(["did","label"])
            
print(strs.collect())
labs.show()


[((0, 'this'), 1), ((0, 'is'), 1), ((0, 'something'), 1), ((0, 'i'), 1), ((0, 'really'), 1), ((0, 'want'), 1), ((0, 'to'), 1), ((0, 'test'), 1), ((1, 'to'), 1), ((1, 'test'), 1), ((1, 'something'), 1), ((1, 'is'), 1), ((1, 'something'), 1), ((1, 'i'), 1), ((1, 'really'), 1), ((1, 'want'), 1), ((1, 'to'), 1), ((1, 'do'), 1), ((2, 'i'), 1), ((2, 'need'), 1), ((2, 'to'), 1), ((2, 'do'), 1), ((2, 'something'), 1), ((2, 'that'), 1), ((2, 'i'), 1), ((2, 'want'), 1), ((2, 'to'), 1), ((2, 'do'), 1), ((2, 'which'), 1), ((2, 'is'), 1), ((2, 'to'), 1), ((2, 'test'), 1), ((2, 'something'), 1)]
+---+-----+
|did|label|
+---+-----+
|  1|    1|
|  2|    0|
|  3|    1|
+---+-----+



In [7]:
sstrs = strs.reduceByKey(add)\
            .map(lambda x: (x[0][1],(x[0][0],x[1])))
    
vocab = sstrs.keys().distinct().zipWithIndex()

sstrs = sstrs.join(vocab).map(lambda x: (x[1][0][0],x[1][1],x[1][0][1]))
print(sstrs.sortBy(lambda x: x[0]).collect())

sstrs = sstrs.map(lambda x: MatrixEntry(x[0],x[1],x[2]))
mat = CoordinateMatrix(sstrs).toIndexedRowMatrix().rows.toDF(["did","features"])
mat.show()

[(0, 0, 1), (0, 2, 1), (0, 3, 1), (0, 8, 1), (0, 4, 1), (0, 9, 1), (0, 10, 1), (0, 11, 1), (1, 0, 1), (1, 2, 2), (1, 3, 1), (1, 1, 1), (1, 4, 2), (1, 9, 1), (1, 10, 1), (1, 11, 1), (2, 0, 2), (2, 2, 2), (2, 5, 1), (2, 6, 1), (2, 7, 1), (2, 1, 2), (2, 4, 3), (2, 9, 1), (2, 10, 1), (2, 11, 1)]
+---+--------------------+
|did|            features|
+---+--------------------+
|  0|(12,[0,2,3,4,8,9,...|
|  1|(12,[0,1,2,3,4,9,...|
|  2|(12,[0,1,2,4,5,6,...|
+---+--------------------+



In [10]:
fin = mat.join(labs,mat.did==labs.did,how="inner")
fin.show()

+---+--------------------+---+-----+
|did|            features|did|label|
+---+--------------------+---+-----+
|  1|(12,[0,1,2,3,4,9,...|  1|    1|
|  2|(12,[0,1,2,4,5,6,...|  2|    0|
+---+--------------------+---+-----+

